In [1]:
import skimage.io

import itertools
import sys

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.integrate

# Import Altair for high level plotting
import altair as alt
import altair_catplot as altcat

# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.plotting

import bebi103

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

# Pevent bulky altair plots
alt.data_transformers.enable('json')

# A whole bunch of skimage stuff
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation


/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/viz.py:30: UserWarning: DataShader import failed with error "No module named 'datashader'".
Features requiring DataShader will not work and you will get exceptions.
  Features requiring DataShader will not work and you will get exceptions.""")
/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/viz.py:38: UserWarning: Could not import `stan` submodule. Perhaps pystan is not properly installed.
  warnings.warn('Could not import `stan` submodule. Perhaps pystan is not properly installed.')
/Users/maria/anaconda/lib/python3.6/site-packages/bebi103/__init__.py:19: UserWarning: Could not import `stan` submodule. Perhaps pystan is not properly installed.
  warnings.warn('Could not import `stan` submodule. Perhaps pystan is not properly installed.')


Loading BokehJS ...

In [2]:
# Load in TIFF stack
fname = '../data/goehring_FRAP_data/PH_138_A.tif'
ic = skimage.io.ImageCollection(fname, conserve_memory=False)[0]

# How long is it?
print('There are {0:d} frames.'.format(len(ic)))

There are 128 frames.


In [3]:
ic[0]

array([ 371,  489,  497,  726,  800, 1344,  711,  555,  750,  552,  371,
        682,  446, 1415, 1084,  407,  294,  426,  775, 1255,  777,  754,
        300,  791,  882, 1844,  916,  992,  750,  329,  674,  610,  504,
        649, 1039,  687,  172,  369,  937,  566,  664,  847,  872,  679,
        615, 1743,  860,  740,  876,  499,  953,  814, 2102,  863,  752,
        810,  867, 1240,  548,  686,  880,  661,  886,  520,  604,  666,
        569,  338, 1140,  927,  566,  522,  664, 1140,  683,  573,  805,
        780, 1006,  796,  723, 1434,  639,  509,  479,  450,  307,  696,
        679,  600, 1392,  509,  756,  638,  430,  963, 1090,  853,  709,
        325,  822,  773,  722,  282,  987,  568,  800, 1288,  547, 1266,
        417,  522,  961, 1193,  785,  422,  402,  650,  414,  306,  511,
        568,  294,  301,  436,  711,  294,  100], dtype=uint16)

- identify the bleached region (will stay constant)
- pull out average intensity of that region over the whle video
- divide that video by the inital average intensity

- look for the darkest pixel in frame 21 (first postbleach pixel)
- pass a 40x40 mean filter though the image and do something?

## Problem 3.2.b

The files bacterium_1.tif and bacterium_2.tif are TIFF stacks of time courses for specific cells in from the full field images. From these time courses, determine the bacterial area for each time point. Assign each time point with an identifier for which division it is. This means you need to determine when divisions happen.

In [4]:
def show_two_ims(im_1, im_2, titles=[None, None],
                 interpixel_distances=[0.065, 0.065], color_mapper=None):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.viz.imshow(im_1,
                             plot_height=300,
                             title=titles[0],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units='µm')
    p_2 = bebi103.viz.imshow(im_2,
                             plot_height=300,
                             title=titles[1],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[1],
                             length_units='µm')
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

In [5]:
# Define directory containing files
im_file = '../data/iyer_biswas_et_al/bacterium_1.tif'

# So we have it, the interpixel distance
ip = 0.052  # microns

# Get image file
im = skimage.io.imread(im_file)

In [6]:
print(type(im), im.dtype, im.shape)
len(im)

<class 'numpy.ndarray'> uint16 (1900, 100, 150)


1900

In [7]:
# Display raw images
plot = bebi103.viz.imshow(im[18],
                            plot_height=300,
                            title=im_file,
                            interpixel_distance=ip,
                            length_units='µm')
bokeh.io.show(plot)

/Users/maria/anaconda/lib/python3.6/site-packages/bokeh/core/json_encoder.py:73: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  if pd and isinstance(obj, pd.tslib.Timestamp):
/Users/maria/anaconda/lib/python3.6/site-packages/bokeh/core/json_encoder.py:75: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(obj), np.float):


First, I tried using Canny Edge Detection. However, since we first find the edges and then fill the shapes in, we join the cells back together which isn't our final goal. The result is suseptible to noise. In our case, cells were not shaded in accurately enough for our needs, so we abondoned this method in favor of the Laplacian of Gaussian and zero crossing method.

In [11]:
# Convert image to float
im_float = (im[100].astype(float) - im[100].min()) / (im[100].max() - im[100].min())

# Do Canny edge detection on image, use sigma = 1.4
im_edge = skimage.feature.canny(im_float, 1.4)

# Show the result
# bokeh.io.show(bebi103.viz.imshow(im_edge))

bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

In [9]:
# Fill the holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Look at result
bokeh.io.show(show_two_ims(im_float,
                           im_bw,
                           titles=['original', 'segmented']))


In [10]:
# Close the edge image
selem = skimage.morphology.disk(2)
im_edge_closed = skimage.morphology.binary_closing(im_edge, selem)

# Fill these holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge_closed)

# Check out results
bokeh.io.show(show_two_ims(im_float,
                           im_bw,
                           titles=['original', 'segmented']))

In [13]:
# Compute LoG
im_LoG = scipy.ndimage.filters.gaussian_laplace(im_float, 2.0)

# Check out results
bokeh.io.show(
    bebi103.viz.imshow(im_LoG,
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('bwr'),
                       interpixel_distance=ip,
                       length_units='µm',
                       colorbar=True))

In [15]:
# 3x3 square structuring element
selem = skimage.morphology.square(3)
    
# Do max filter and min filter
im_LoG_max = scipy.ndimage.filters.maximum_filter(im_LoG, footprint=selem)
im_LoG_min = scipy.ndimage.filters.minimum_filter(im_LoG, footprint=selem)

# Image of zero-crossings
im_edge = (  ((im_LoG >= 0) & (im_LoG_min < 0)) 
           | ((im_LoG <= 0) & (im_LoG_max > 0)))

# Show result
bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

In [16]:
def zero_crossing_filter(im, thresh):
    """
    Returns image with 1 if there is a zero crossing and 0 otherwise.
    
    thresh is the the minimal value of the gradient, as computed by Sobel
    filter, at crossing to count as a crossing.
    """
    # Square structuring element
    selem = skimage.morphology.square(3)
    
    # Do max filter and min filter
    im_max = scipy.ndimage.filters.maximum_filter(im, footprint=selem)
    im_min = scipy.ndimage.filters.minimum_filter(im, footprint=selem)
    
    # Compute gradients using Sobel filter
    im_grad = skimage.filters.sobel(im)
    
    # Return edges
    return ( (  ((im >= 0) & (im_min < 0))
              | ((im <= 0) & (im_max > 0))) 
            & (im_grad >= thresh) )

In [17]:
# Find zero-crossings
im_edge = zero_crossing_filter(im_LoG, 0.001)

# Show result
bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

In [18]:
# Skeletonize edges
im_edge = skimage.morphology.skeletonize(im_edge)

# See result
bokeh.io.show(show_two_ims(im_float,
                           im_edge,
                           titles=['original', 'edges']))

In [20]:
# Fill holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge)

# Remove small objectes that are not bacteria
im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=100)

# Show result
bokeh.io.show(show_two_ims(im_float,
                           im_bw,
                           titles=['original', 'segmented']))


In [21]:
# Clear border with large buffer size b/c LoG procedure came off border
im_bw = skimage.segmentation.clear_border(im_bw, buffer_size=5)

# Show result
bokeh.io.show(
    bebi103.viz.imshow(im_bw, 
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('gray'),
                       interpixel_distance=ip,
                       length_units='µm'))

In [22]:
# Label binary image; backward kwarg says value in im_bw to consider backgr.
im_labeled, n_labels = skimage.measure.label(
                            im_bw, background=0, return_num=True)

# Show number of bacteria
print('Number of individual bacteria = ', n_labels - 1)

Number of individual bacteria =  0


- pick the largest area
- pick the area nearest to the previous center
- pick the rightmost area

- first segment, then run that function to get the number of pixels in each chunk